### FLiBe Simple

In [4]:
import openmc
import os
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# Helper functions
def logspace_per_decade(start, stop, pts_per_decade):
    """
    Returns values from 'start' to 'stop' so that each factor-of-10
    interval contains 'pts_per_decade' points (including its first endpoint).
    Might be a little off if 'stop' isn't precisely at a decade, ex. 20e6 eV

    example: 10 points per decade from 1e-5 → 2e7
    grid = logspace_per_decade(1e-5, 20e6, pts_per_decade=10)
    for i in grid:
        print(np.log10(i))
    # print(np.log10(i) for i in grid)
    """
    log_start = np.log10(start)
    log_stop  = np.log10(stop)
    total_decades = log_stop - log_start
    # how many intervals of size 1 decade we need, as a float
    total_steps = total_decades * pts_per_decade
    # +1 so that we include the very last point at `stop`
    npts = int(np.ceil(total_steps)) + 1
    return np.logspace(log_start, log_stop, num=npts)

Neutronics codes like MCNP and OpenMC define some model on an input deck or file. This input has three main components: Materials, Geometry, and Problem Settings. 
### Materials
We define the composition . Each `Material` class is given some density, temperature, and isotopic composition. Materials can be (homogeneously) mixed to create a new material. The materials we actually use in the problem are then collected into the `Materials` class and exported as an `.xml` file.

In [7]:
# User specifications
DENSITY_FLIBE = 1.8 # usu 1.8-2.0 g/cc
ENRICH_LI = 20
MOL_LIF, MOL_BEF2 = 0.66, 0.34
TEMP = 900 # K

VOL = 342 * 100e3 # cm3
M_U, M_FLIBE = 0.0181*20/50, 1 # in Ball et al. --EZ
MR_U, MR_FLIBE = M_U / (M_U + M_FLIBE), M_FLIBE / (M_U + M_FLIBE) # mol ratios

""" 
MATERIALS
  OpenMC automatically normalizes the fraction of each element in material (like MCNP)
  but the fractions in 'mix_materials' MUST add up to 1
"""
flibe = openmc.Material(name="FLiBe", temperature=TEMP)
flibe.set_density('g/cm3', DENSITY_FLIBE) 
flibe.add_element('Li',MOL_LIF,'ao',ENRICH_LI,'Li6','wo')
flibe.add_element('Be',MOL_BEF2,'ao')
flibe.add_element('F',(MOL_LIF+2*MOL_BEF2),'ao')

uranium = openmc.Material(name="U",temperature=TEMP)
uranium.add_element('U',100,'ao',0.7204)
# uranium.add_nuclide('U238',100) --ezoccoli

mix = openmc.Material.mix_materials([flibe, uranium], [MR_FLIBE, MR_U], 'ao') # 

# print(flibe.get_nuclide_atom_densities()) # Returns one or all elements in the material and their atomic densities in units of atom/b-cm
# print(f"ppark checksum for FLiBe 1.8 g/cc, 0.64 mol LiF, 0.33 mol BeF2:")
# print(f"  'Li6': 0.00489411249582549, 'Li7': 0.016783401745946593, 'Be9': 0.011167204306367438, 'F19': 0.04401192285450695")

materials = openmc.Materials([mix])
materials.cross_sections = '/mnt/c/OpenMC/data/endfb-viii.0-hdf5/cross_sections.xml'
os.makedirs("./xml/", exist_ok=True) 
materials.export_to_xml("./xml/materials.xml")

#### Geometry
For our simple case suppose we make a 1 m$^3$ cube with reflective surfaces of homogeneous FLiBe. We define a cubical cell defined by 6 planes, and place this cell in the "universe." (Alex actually suggested switching from 1 cm side to 1 m side because if the geometry is too small we'll spend too much computational time calculating neutron reflections.) Then, like the materials, we export these definitions to `geometry.xml`.

In [8]:
""" GEOMETRY
"""
xmin = openmc.XPlane(-50, boundary_type='reflective')
xmax = openmc.XPlane( 50, boundary_type='reflective')
ymin = openmc.YPlane(-50, boundary_type='reflective')
ymax = openmc.YPlane( 50, boundary_type='reflective')
zmin = openmc.ZPlane(-50, boundary_type='reflective')
zmax = openmc.ZPlane( 50, boundary_type='reflective')

region = +xmin & -xmax & +ymin & -ymax & +zmin & -zmax
cell = openmc.Cell(fill=mix, region=region)
root_univ = openmc.Universe(cells=[cell])

geometry = openmc.Geometry(root_univ)
geometry.export_to_xml("./xml/geometry.xml")

#### Settings
Here we define the other settings for our problem. 

In [10]:
""" 
SETTINGS
"""
settings = openmc.Settings()

""" Tallies
"""
# particle_filter = openmc.ParticleFilter(['neutron'])
energy_filter = openmc.EnergyFilter(np.linspace(1e-5, 20e6, 719))
# energy_filter = openmc.EnergyFilter.from_group_structure('CCFE-709')
# These have extra bins in key energy ranges. A full list of energy structures is available here:
# https://github.com/openmc-dev/openmc/blob/6254be37582e09acff038f5656332b89e53e4eae/openmc/mgxs/__init__.py#L50-L420

# cell_filter = openmc.CellFilter([cell]) 

# (n,gamma) Tally
capture_tally = openmc.Tally(name='capture tally')
capture_tally.scores = ['(n,gamma)',] # specific names required
capture_tally.nuclides = ['U238',]
capture_tally.filters = [energy_filter]

tallies = openmc.Tallies([capture_tally])
# tallies.append(capture_tally)
tallies.export_to_xml("./xml/tallies.xml")

""" 
Source
  Isotropic 14.07 MeV point source at center of cube
"""
source = openmc.IndependentSource()
source.space = openmc.stats.Point((0,0,0))
source.angle = openmc.stats.Isotropic()
source.energy = openmc.stats.Discrete([14.07e6], [1.0])
settings.source = source

"""
Run type
"""
settings.run_mode = 'fixed source'
settings.particles = int(1e5)
settings.generations_per_batch = 10
settings.batches = 100
settings.inactive = 5

settings.export_to_xml("./xml/settings.xml")

Run OpenMC

In [11]:
# if __name__ == '__main__':
#     # If you were to make this a .py script then you should place everything below underneath this if statement
openmc.run(cwd="./xml")

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################